<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="200" height="200">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="200" height="200">
# Autoencoder Variable Transformation
<hr style="border-top-width: 4px; border-top-color: #34609b;"> 
Notebook by Abhinav Moudgil and Neural Network Visualisation by Attila Bagoly

In [1]:
import sys, os
PATH_TO_JSMVA_FOLDER="/Users/testuser/gsoc-2016/jsMVA/src/python/"
sys.path.append(os.path.expanduser(PATH_TO_JSMVA_FOLDER))
import JsMVA

In [2]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.07/07


In [3]:
%jsmva on

In [4]:
outputFile = TFile("TMVAOutput.root", "RECREATE")
inputFile  = TFile("../datasets/mydataset.root")

TMVA.Tools.Instance()

factory = TMVA.Factory("TMVAClassification",
                       outputFile,
                       "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification")
   
loader = TMVA.DataLoader("mydataset")

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [5]:
# Adding variables to dataset
loader.AddVariable("var0", 'F')
loader.AddVariable("var1", 'F')
loader.AddVariable("var2", 'F')
loader.AddVariable("var3 := var0-var1", 'F')
loader.AddVariable("var4 := var0*var2", 'F')
loader.AddVariable("var5 := var1+var2", 'F')

In [6]:
print "--- TMVAClassification       : Using input file:", inputFile.GetName()
   
# Register the training and test trees

tsignal     = inputFile.Get("MyMCSig")
tbackground = inputFile.Get("MyMCBkg")
     
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(tsignal, signalWeight)
loader.AddBackgroundTree(tbackground, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = tsignal
loader.fTreeB = tbackground
loader.PrepareTrainingAndTestTree(mycuts,
                                  mycutb,
                                  "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V")

--- TMVAClassification       : Using input file: ../datasets/mydataset.root
--- DataSetInfo              : Dataset[mydataset] : Added class "Signal"	 with internal class number 0
--- mydataset                : Add Tree MyMCSig of type Signal with 5449 events
--- DataSetInfo              : Dataset[mydataset] : Added class "Background"	 with internal class number 1
--- mydataset                : Add Tree MyMCBkg of type Background with 5449 events
--- mydataset                : Preparing trees for training and testing...


In [7]:
loader.CalcNorm()
# factory.BookMethod(loader,
#                    TMVA.Types.kMLP,
#                    "MLP",
#                    "!V:NeuronType=tanh:VarTransform=N:NCycles=150:HiddenLayers=N+5:TestRate=5:!UseRegulator")

--- DataSetFactory           : Dataset[mydataset] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[mydataset] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[mydataset] :     Signal          -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Background      -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Signal     tre

In [8]:
newloader = loader.VarTransform("AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|3,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1)")

--- mydataset                : [AE Transform] indexLayer = 1
--- mydataset                : [AE Transform] Pretraining = false
--- mydataset                : Preparing DataLoader for Autoencoder Transform DNN Training
--- DataSetInfo              : Dataset[ae_transform_dataset] : Added class "Regression"	 with internal class number 0
--- ae_transform_dataset     : Add Tree Dataset of type Regression with 6000 events
--- DataSetInfo              : Dataset[ae_transform_dataset] : Class index : 0  name : Regression
--- DNN                      : Parsing option string: 
--- DNN                      : ... "!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|3,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,Batc

In [9]:
factory.DrawNeuralNetwork("ae_transform_dataset", "DNN", True)